In [1]:
#import necessary libraries
import pandas as pd
import numpy as np

In [2]:
#load the training dataset
df = pd.read_csv('../data/train.csv')

In [3]:
#get data head
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
#get data info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
#get data statistics
df.describe().T

,count,mean,std,min,25%,50%,75%,max
PassengerId,891.0,446.000000,257.353842,1.00,223.5000,446.0000,668.5,891.0000
Survived,891.0,0.383838,0.486592,0.00,0.0000,0.0000,1.0,1.0000
Pclass,891.0,2.308642,0.836071,1.00,2.0000,3.0000,3.0,3.0000
Age,714.0,29.699118,14.526497,0.42,20.1250,28.0000,38.0,80.0000
SibSp,891.0,0.523008,1.102743,0.00,0.0000,0.0000,1.0,8.0000
Parch,891.0,0.381594,0.806057,0.00,0.0000,0.0000,0.0,6.0000
Fare,891.0,32.204208,49.693429,0.00,7.9104,14.4542,31.0,512.3292


In [6]:
#find number of missing values in each column (along with percentages)
missing_values = df.isnull().sum()
missing_percentages = (missing_values / len(df)) * 100
missing_data = pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_percentages})
missing_data = missing_data[missing_data['Missing Values'] > 0]
print(missing_data)

          Missing Values  Percentage
Age                  177   19.865320
Cabin                687   77.104377
Embarked               2    0.224467


In [7]:
"""
Since there are only 2 rows with missing 'Embarked' values, we can drop these rows without significant loss of information.

Since around 20% of the 'Age' values are missing, we can consider imputing these missing values using the median age or by using more advanced techniques later in the analysis.

Since almost 77% of the 'Cabin' values are missing, we can drop this column altogether for now.
"""
#drop rows with missing 'Embarked' values
df = df.dropna(subset=['Embarked'])

#drop 'Cabin' column
df = df.drop(columns=['Cabin'])

In [8]:
#identify number of unique values in each column
unique_values = df.nunique()
print(unique_values)

PassengerId    889
Survived         2
Pclass           3
Name           889
Sex              2
Age             88
SibSp            7
Parch            7
Ticket         680
Fare           247
Embarked         3
dtype: int64


In [9]:
"""
Since there are only 680 unique tickets, but 889 tickets in total, it indicates that some tickets were shared among multiple passengers.
Let us create a new feature indicating the group size based on the ticket number.
"""
#calculate group size based on ticket number
df['GroupSize'] = df.groupby('Ticket')['Ticket'].transform('count')

In [10]:
#get a quick overview of the dataset
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,GroupSize
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,2
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,1


In [11]:
#check if any group size is missing
print(df['GroupSize'].isnull().sum())

0


In [12]:
#drop original 'Ticket' column after creating 'GroupSize'
df.drop(columns=['Ticket'], inplace=True)

In [13]:
"""
The feature 'Name' in itself is useless, but it indicates the title of each passenger (Mr., Mrs., Miss, etc.) which can be useful. 
We will extract the titles from the names and create a new feature 'Title'.
"""
#extract titles from names
df['Title'] = df['Name'].str.extract(r' ([A-Za-z]+)\.', expand=False)

In [14]:
#check the unique titles extracted
print(df['Title'].unique())

['Mr' 'Mrs' 'Miss' 'Master' 'Don' 'Rev' 'Dr' 'Mme' 'Ms' 'Major' 'Lady'
 'Sir' 'Mlle' 'Col' 'Capt' 'Countess' 'Jonkheer']


In [15]:
#look for missing titles
print(df['Title'].isnull().sum())

0


In [16]:
#drop original 'Name' column after extracting 'Title'
df.drop(columns=['Name'], inplace=True)

In [17]:
#check count of each title
print(df['Title'].value_counts())

Title
Mr          517
Miss        181
Mrs         124
Master       40
Dr            7
Rev           6
Col           2
Mlle          2
Major         2
Ms            1
Mme           1
Don           1
Lady          1
Sir           1
Capt          1
Countess      1
Jonkheer      1
Name: count, dtype: int64


In [18]:
#reduce title categories by grouping titles together

#group common titles
df['Title'] = df['Title'].replace({'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs'})
#group rare titles
df['Title'] = df['Title'].replace(['Don', 'Rev', 'Dr', 'Major', 'Lady', 'Sir', 'Col', 'Capt', 'Countess', 'Jonkheer'], 'Rare')

In [19]:
#check the unique titles and their counts after grouping
print(df['Title'].value_counts())

Title
Mr        517
Miss      184
Mrs       125
Master     40
Rare       23
Name: count, dtype: int64


In [20]:
#get a quick overview of the dataset
df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,GroupSize,Title
0,1,0,3,male,22.0,1,0,7.2500,S,1,Mr
1,2,1,1,female,38.0,1,0,71.2833,C,1,Mrs
2,3,1,3,female,26.0,0,0,7.9250,S,1,Miss
3,4,1,1,female,35.0,1,0,53.1000,S,2,Mrs
4,5,0,3,male,35.0,0,0,8.0500,S,1,Mr


In [21]:
"""
It seems 'Fare' has floating point values. We can convert them to nearest integers for simplicity.
"""
#convert 'Fare' to nearest integers
df['Fare'] = df['Fare'].round().astype(int)

In [22]:
#count unique fares
print(df['Fare'].nunique())

90


In [23]:
#get data info after modifications
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 889 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  889 non-null    int64  
 1   Survived     889 non-null    int64  
 2   Pclass       889 non-null    int64  
 3   Sex          889 non-null    object 
 4   Age          712 non-null    float64
 5   SibSp        889 non-null    int64  
 6   Parch        889 non-null    int64  
 7   Fare         889 non-null    int64  
 8   Embarked     889 non-null    object 
 9   GroupSize    889 non-null    int64  
 10  Title        889 non-null    object 
dtypes: float64(1), int64(7), object(3)
memory usage: 83.3+ KB


In [24]:
#impute age based on the median age of passengers with the same title, class and sex
df['Age'] = df.groupby(['Title', 'Pclass', 'Sex'])['Age'].transform(lambda x: x.fillna(x.median()))

In [25]:
#check for any remaining missing 'Age' values
print(df['Age'].isnull().sum())

0


In [26]:
#save the cleaned and modified dataset to a new CSV file
df.to_csv('../data/train_partial_fe.csv', index=False)